In [1]:
! pip3 install --upgrade --quiet google-cloud-aiplatform openai gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [2]:
! pip3 install gtts

In [1]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

In [2]:
PROJECT_ID = "lexical-pattern-439216-d8"  # @param {type:"string"}

# Only `us-central1` is supported region for Llama 3.2 models using Model-as-a-Service (MaaS).
LOCATION = "us-central1"

In [3]:
BUCKET_NAME = "llama3_2_example_bucket"  # @param {type:"string"}

BUCKET_URI = f"gs://{BUCKET_NAME}"

In [4]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [7]:
credentials, _ = default()
auth_request = transport.requests.Request()
credentials.refresh(auth_request)

In [8]:
MODEL_LOCATION = "us-central1"
MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

client = openai.OpenAI(
    base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
    api_key=credentials.token,
)

In [9]:
MODEL_ID = "meta/llama-3.2-90b-vision-instruct-maas"  # @param {type:"string"} ["meta/llama-3.2-90b-vision-instruct-maas"]

In [15]:
max_tokens = 4096

response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "image_url": {
                        "url": "gs://llama3_2_example_bucket/women dressed casually.webp"
                    },
                    "type": "image_url",
                },
                {"text": "What’s in this image? Can you describe what she is wearing", "type": "text"},
            ],
        },
        {"role": "assistant", "content": "You are a fashion assistant and you analyse what a person is wearing in the image. In this image, you have:"},
    ],
    max_tokens=max_tokens,
)

In [14]:
print(response.choices[0].message.content)

The person in the image is wearing a white t-shirt, light-blue jeans, and white sneakers.


In [24]:
import cv2
import os
import uuid
import tempfile
from google.cloud import storage
from gtts import gTTS
from google.auth.transport import requests
from google.auth import default
import openai
import gradio as gr
import asyncio
import aiohttp
import io
from concurrent.futures import ThreadPoolExecutor
import time

max_tokens = 50  # Adjusted for shorter commentary

async def upload_file_to_bucket(file_path: str) -> str:
    """Uploads a file to a Google Cloud Storage bucket."""
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(BUCKET_NAME)

        file_extension = file_path.split(".")[-1].lower()
        blob_name = f"analysis_{uuid.uuid4()}.{file_extension}"

        blob = bucket.blob(blob_name)
        await asyncio.to_thread(blob.upload_from_filename, file_path)

        file_uri = f"gs://{BUCKET_NAME}/{blob_name}"
        return file_uri
    except Exception as e:
        print(f"Error uploading file: {e}")
        return None

async def process_frame(image_uri):
    """Processes a video frame using a multimodal LLM."""
    prompt = """
    You're a cricket commentator. Describe the current action in one short, exciting sentence (under 10 words).
    Focus on key moments and player actions. Keep it brief and engaging.
    Keep it flowing.
    It is fine to send an empty string as response if the frame doesnt have a any crucial information to share.
    """

    credentials, _ = await asyncio.to_thread(default)
    auth_request = requests.Request()
    await asyncio.to_thread(credentials.refresh, auth_request)

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )
    response = await asyncio.to_thread(
        client.chat.completions.create,
        model=MODEL_ID,
        messages=[
            {
                "role": "user",
                "content": [
                    {"image_url": {"url": image_uri}, "type": "image_url"},
                    {"text": prompt, "type": "text"},
                ],
            },
        ],
        max_tokens=max_tokens,
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content

async def text_to_speech(text, lang='en'):
    """Converts text to speech using gTTS and saves it as an MP3 file."""

    # Create a gTTS object with the given text and language
    tts = gTTS(text=text, lang=lang)

    # Save the generated speech to a file
    file_path = "output.mp3"
    tts.save(file_path)

    # Since gTTS is synchronous, simulate async behavior
    await asyncio.sleep(0)  # Simulating an async non-blocking operation

    # Read the saved audio content
    with open(file_path, "rb") as audio_file:
        audio_content = audio_file.read()

    return audio_content

async def process_video_stream(video_path):
    """Processes a video stream, generating and yielding commentary with audio and video frames."""
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * 3) // 2  # Process every half second

    frame_count = 0
    tasks = []

    with ThreadPoolExecutor() as executor:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % frame_interval == 0:
                temp_frame_path = f"temp_frame_{frame_count}.jpg"
                cv2.imwrite(temp_frame_path, frame)

                frame_uri = await upload_file_to_bucket(temp_frame_path)
                task = asyncio.create_task(process_frame(frame_uri))
                tasks.append(task)

                os.remove(temp_frame_path)

            frame_count += 1

            # Process completed tasks
            if tasks:  # Only wait if there are tasks
                done, pending = await asyncio.wait(tasks, timeout=0.1, return_when=asyncio.FIRST_COMPLETED)
                tasks = list(pending)  # Convert set to list
                for task in done:
                    commentary = task.result()
                    audio_data = await text_to_speech(commentary)

                    if audio_data:
                        # Save audio to a temporary file
                        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
                            temp_audio_file.write(audio_data)
                            temp_audio_path = temp_audio_file.name

                        yield commentary, temp_audio_path, frame
            else:
                # If no tasks are processing, just yield the frame
                yield None, None, frame

            # Simulate real-time processing
            await asyncio.sleep(1 / fps)

    cap.release()

    # Process any remaining tasks
    for task in asyncio.as_completed(tasks):
        commentary = await task
        audio_data = await text_to_speech(commentary)

        if audio_data:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
                temp_audio_file.write(audio_data)
                temp_audio_path = temp_audio_file.name

            yield commentary, temp_audio_path, None

async def demo_fn(video_path):
    """
    Processes a video using a multimodal LLM and generates live commentary with audio and video playback.
    """
    async for commentary, audio_path, frame in process_video_stream(video_path):
        if frame is not None:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        yield commentary, audio_path, frame

# Gradio interface
demo = gr.Interface(
    fn=demo_fn,
    inputs=gr.File(label="Upload Cricket Video"),
    outputs=[
        gr.Textbox(label="Live Commentary"),
        gr.Audio(label="Audio Commentary", streaming=True),
        gr.Image(label="Video Playback", streaming=True)
    ],
    title="Live Cricket Commentary with AI",
    live=True
)


In [ ]:
demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d35b6d7c6a03e1020b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The batter swings hard, unleashing a powerful shot.
Babar smashes a six to the boundary.
The batsman is hitting a shot.
The bowler is delivering the ball.
Bowled, oh what a delivery.
The bowler is bowling the ball.
Babar unleashes a powerful shot to the boundary.
Bowled, he's gone for a duck.
The bowler is delivering the ball.
The batsman swings his bat with all his might.
The batsman is in mid-swing, hitting the ball.
The batsman is about to hit the ball.
The bowler is about to throw the ball.
The ball is heading straight for the batsman.
Babar smashes a boundary to the mid-wicket region.
The ball is flying through the air.
